In [1]:

from pandas import Series, DataFrame
import pandas as pd
from patsy import dmatrices

%pylab inline
import re
import string

Populating the interactive namespace from numpy and matplotlib


In [2]:
#comments are coming in as utf-8 so I am using this code to change it back to ascii
def unicodetoascii(text):

    TEXT = (text.
    		replace('\xe2\x80\x99', "'").
            replace('\xc3\xa9', 'e').
            replace('\xe2\x80\x90', '-').
            replace('\xe2\x80\x91', '-').
            replace('\xe2\x80\x92', '-').
            replace('\xe2\x80\x93', '-').
            replace('\xe2\x80\x94', '-').
            replace('\xe2\x80\x94', '-').
            replace('\xe2\x80\x98', "'").
            replace('\xe2\x80\x9b', "'").
            replace('\xe2\x80\x9c', '"').
            replace('\xe2\x80\x9c', '"').
            replace('\xe2\x80\x9d', '"').
            replace('\xe2\x80\x9e', '"').
            replace('\xe2\x80\x9f', '"').
            replace('\xe2\x80\xa6', '...').#
            replace('\xe2\x80\xb2', "'").
            replace('\xe2\x80\xb3', "'").
            replace('\xe2\x80\xb4', "'").
            replace('\xe2\x80\xb5', "'").
            replace('\xe2\x80\xb6', "'").
            replace('\xe2\x80\xb7', "'").
            replace('\xe2\x81\xba', "+").
            replace('\xe2\x81\xbb', "-").
            replace('\xe2\x81\xbc', "=").
            replace('\xe2\x81\xbd', "(").
            replace('\xe2\x81\xbe', ")")

                 )
    return TEXT

In [3]:
data = pd.read_csv('final.csv', sep=',',names=['id', 'date','user','text'])

In [4]:
data.drop(data.index[:1], inplace=True)


In [ ]:
data

In [5]:
#text field is the comments field
#unicodetoascii changes the form from unicoding to regular english
#making all the comments lower case
#dt = data.iloc[:50]
text = data['text']
text = text.map(lambda a: unicodetoascii(str(a)))
text = text.map(lambda a: a.lower())

#text= text.map(lambda a: str(a).decode('ascii', errors='ignore'))
#text_18 = text.loc[18]

#text_18 = text_18.decode("ascii", errors="ignore")
#text_18
text

1       if they keep it around in next four-five years...
2       the lease rate is the factor that stops me col...
3       yes, the completely noncomparable m2 would bea...
4                    why not a genesis g70 with a manual?
5       again, the local dealer are hopeless at best(w...
6       are you talking about genesis dealers or hyund...
7       thanks, but i am more than aware of the distin...
8       yeah, i went once to an autonation lexus store...
9       i am the poster formerly known as benjaminh an...
10      still on the list.  just have not been looking...
11                                   i sent you a message
12      a new and mostly positive review by kelly blue...
13      if i do manage to upgrade to a tlx it would ha...
14      thanks. i'll give it a try at some point to me...
15      i'm not sure if i'd have the patience and disc...
16      they usually go up as the year goes along. chr...
17      yeah, i was and am a bit concerned with gettin...
18      the kb

In [6]:
models = pd.read_csv('models.csv', sep=',',names=['brand','model'])

In [7]:
#this text is also in unicode so just replacing one of the symbols with a space (we can also call the unicodetoascii function here for more robust cleaning)
#making the model names lowercase
models.model = models['model'].map(lambda x: x.replace('\xa0', ''))
models.model = models['model'].map(lambda x: x.lower())


In [ ]:
models

In [8]:
#create dictionary {model: brand}
#this will be used when locating when the model is written, and replacing it to the brand name
brand_dict = {}
for i in range(len(models)):
    brand_dict[models['model'][i]] = models['brand'][i]

In [ ]:
brand_dict

In [9]:
#replace all models with brand names


#for key in brand_dict.iterkeys():
 #   dt['text'] = dt['text'].map(lambda x: x.replace(key, brand_dict[key]))
    
#dt['text']
#for key in brand_dict.iterkeys():
#    text_18 = text_18.replace(key, brand_dict[key])
#text_18

def replace(match):
    return brand_dict[match.group(0)]

text = text.map(lambda x: re.sub('|'.join(r'\b%s' % re.escape(s) for s in brand_dict), 
        replace, x) )

#text_18 =  re.sub('|'.join(r'\b%s' % re.escape(s) for s in brand_dict), 
#        replace, text_18) 
text

1       if they keep it around in next four-five years...
2       the lease rate is the factor that stops me col...
3       yes, the completely noncomparable m2 would bea...
4                    why not a hyundai g70 with a manual?
5       again, the local dealer are hopeless at best(w...
6       are you talking about hyundai dealers or hyund...
7       thanks, but i am more than aware of the distin...
8       yeah, i went once to an autonation lexus store...
9       i am the poster formerly known as benjaminh an...
10      still on the list.  just have not been looking...
11                                   i sent you a message
12      a new and mostly positive review by kelly blue...
13      if i do manage to upgrade to a tlx it would ha...
14      thanks. i'll give it a try at some point to me...
15      i'm not sure if i'd have the patience and disc...
16      they usually go up as the year goes along. chr...
17      yeah, i was and am a bit concerned with gettin...
18      the kb

In [10]:
#finding all of the unique/different brand names
brand = list(set(models.brand))

In [11]:
brand

['toyata',
 'seat',
 'subaru',
 'buick',
 'audi',
 'mercury',
 'saturn',
 'chevrolet',
 'suzuki',
 'volkswagen',
 'dodge',
 'pontiac',
 'cadillac',
 'honda',
 'hyundai',
 'ford',
 'mazda',
 'hyundai.',
 'mercedes benz',
 'kia,',
 'infiniti',
 'bmw',
 'hyndai kia',
 'mercedes-benz',
 'sedan',
 'kia',
 'mitsubishi',
 'problem',
 'kia.',
 'lincoln',
 'acura',
 'car',
 'nissan',
 'toyota',
 'nissan.',
 'volvo',
 'chrysler',
 'mercedes',
 'hyundai,',
 'volkwagen']

In [12]:
#findall: takes each comment separately, finds every time any brand is mentioned, and adds it to the master list
#set(ls): this makes sure that even if the brand is mentioned more than once, it is only recorded once
master_list = []

def findall(w):
    ls = []
    ls = [e for e in brand for i in w.split() if e in i] #this line finds where the brand is mentioned in the comment (could be
    # of any format: ex. "honda."| "honda's" | "honda-and" | etc., and records it as just "honda")
    ls = list(set(ls))
    master_list.append(ls) #this stores all the mentions of every comment we have
text.map(findall)

master_list = sum(master_list)


In [13]:
from collections import Counter
#counts the number of mentions each brand has and displays the top 10

counts = Counter(master_list)
counts.most_common(10)

[('car', 1019),
 ('bmw', 532),
 ('audi', 404),
 ('toyota', 231),
 ('acura', 217),
 ('honda', 205),
 ('infiniti', 131),
 ('volkswagen', 118),
 ('sedan', 104),
 ('problem', 100)]

In [ ]:
#list = []
#list = [i for e in brand for i in text_18.split() if e in i]
#for b in brand:
#    for c in list:
#        if re.search(b , c):
#            list = [word.replace(c,b) for word in list]
#set(list)